# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?
2. 리뷰어는 어디서 가져오지?
3. 연도를 어떻게 하지 ? 연도가 없는 경우도 있고, 연도와 요일이 없는 경우도 있음 

In [114]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 1. User 정보를 가져오기 위해 (현재 기준 없음) user의 myplace url을 복사한다. 

In [122]:
url = 'https://m.place.naver.com/my/5cf4c309c1dd7fdcdfcd76ac/review?v=2'


User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

In [3]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 
def user_profile(url) :
    # 주소 이동
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['주소'] = url

    print(user_element)

    return user_element


일자, 요일, 방문일수 분류 함수 

In [175]:
def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')
    date = date_part[0]
    weekday = date[-3:]
    revisit = int(date_part[1].replace('번째 방문', ''))

    # 일자 타입 변경하기 
    date = '24' + date if date.startswith('년') else date # 만약 연도가 없는 경우 24를 붙이기
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    year, month, day = map(int, match[0])

    # 날짜 객체로 변환
    date_object = datetime(2000+year, month, day) 
    formatted_date = date_object.strftime('%Y-%m-%d')

    return formatted_date, weekday, revisit



### 유저 정보 찾기

In [146]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
user_data = user_profile(url)


# User 정보 저장 
user_profile_df = pd.DataFrame([user_data], index=[0])
user_profile_df

{'아이디': '포도267', '리뷰': 665, '팔로잉': 0, '팔로워': 13, '주소': 'https://m.place.naver.com/my/5cf4c309c1dd7fdcdfcd76ac/review?v=2'}


,아이디,리뷰,팔로잉,팔로워,주소
0,포도267,665,0,13,https://m.place.naver.com/my/5cf4c309c1dd7fdcd...


In [149]:
# 맨 처음 게시물 클릭 
button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
button.click()
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# data를 담을 빈 리스트 선언
data_list = [] 


### 매장 정보 찾기

In [186]:
# 페이지 아래로 스크롤 몇번 
page_down = 1

# 페이지 스크롤 횟수만큼 반복
for _ in range(page_down):

    # 대기시간 5초 
    wait = WebDriverWait(driver, 5)

    # 요소를 찾을 때 대기 조건 추가
    user_review_elements = wait.until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
    )

    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

    # 리뷰에서 정보 가져오기
    for reviews_elements in user_review_elements:
        i = 0 # 카테고리와 주소 구분할 때 사용
        restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
        
        if bool(restaurant_elements): # 정보가 없는 경우가 있음

            # 매장명 찾기
            restaurant_name = restaurant_elements[0].text  

    #<--------------------------------------------------------------------------------------------------->
            # 카테고리와 주소 찾기 
            category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
            category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
            
            # span 태그 안에 있는 텍스트 가져오기
            span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
            category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
            location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
            i += 1
    #<--------------------------------------------------------------------------------------------------->
            # 리뷰 찾기 
            review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문
            sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
            if bool(sub_info):
                sub_info=sub_info[0].text
            else:
                sub_info=None

    #<--------------------------------------------------------------------------------------------------->
            # 태그 찾기 
            # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
            reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
            command=False
            for x in reactions_elements:
                if x.get_attribute("role"): # 리액션 숨김 없애기
                    x.click()
                    command=True
                else:
                    continue
            if command:
                reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
            reactions=list(map(lambda x:x.text,reactions_elements))
            # 리스트를 벗김 
            reactions = str(reactions).replace('[', '').replace(']', '')

    #<--------------------------------------------------------------------------------------------------->
            # 방문 일자, 재방문 이력 찾기
            date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
            date=[x.text for x in date_info]
            day,weekday,revisit = change_date_format(date) # 함수 사용
        
            # 정보 추가
            data_dict = {
                    '아이디' : user_data['아이디'],
                    '리뷰' : user_data['리뷰'],
                    '팔로워' : user_data['팔로워'],
                    '매장명': restaurant_name,
                    '카테고리': category,
                    '주소' : location,
                    '리뷰 내용': review_text,
                    "세부정보" : sub_info,
                    '태그': reactions,
                    '방문일자': day,
                    '요일' : weekday,
                    '재방문횟수':revisit}
            data_list.append(data_dict)
    user_df = pd.DataFrame(data_list)
        
user_df

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,요일,재방문횟수
0,포도267,665,13,존앤맷 타코 미사본점,"멕시코,남미음식",경기도 하남시 망월동,타히타 계속 생각날 맛 입니다 ! \n미사핫플 멕시코 분위기 담아낸 귀여운 존앤맷 ‼️,,'음식이 맛있어요',2024-02-18,일요일,1
1,포도267,665,13,또바기치킨 길동역점,"치킨,닭강정",서울특별시 강동구 길동,변하지않는 옛닐통닭맛!!,,"'음식이 맛있어요', '재료가 신선해요'",2024-02-17,토요일,1
2,포도267,665,13,보타이,태국음식,서울특별시 강동구 성내동,보타이만한 팟타이 맛집이 없어요 진짜루...,,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요', '현지 맛에 가까워요'",2024-02-15,목요일,7
3,포도267,665,13,그레이우드커피,"카페,디저트",경기도 하남시 망월동,2층이 신기한 구조로 되어있고! 루프탑이 있어서 뷰가 좋아요 :) \n코코넛 커피도...,,"'뷰가 좋아요', '커피가 맛있어요', '음료가 맛있어요'",2024-02-14,수요일,1
4,포도267,665,13,이가네양꼬치 미사점,양꼬치,경기도 하남시 망월동,왕 양꼬치가 있어서 씹는맛이 더 맛있어요!,,"'음식이 맛있어요', '고기 질이 좋아요'",2024-02-14,수요일,1
5,포도267,665,13,맘스터치 굽은다리역점,햄버거,서울특별시 강동구 천호동,친절하고 맛있습니댜 \n싸이버거는 늘..🩷,,'음식이 맛있어요',2024-02-13,화요일,2
6,포도267,665,13,빽다방 미사센트럴자이점,테이크아웃커피,경기도 하남시 풍산동,사장님 항상 친절하시고 일정한 맛에 감사해요,,'음료가 맛있어요',2024-02-13,화요일,6
7,포도267,665,13,신흥정육식당,정육식당,서울특별시 강동구 성내동,더 깔끔해졌어요 갈비살이 최애고 \n상차림비가 빡새져서 놀랐지만 역시 신흥..,,"'음식이 맛있어요', '고기 질이 좋아요', '양이 많아요'",2024-02-12,월요일,4
8,포도267,665,13,빽다방 길동역점,테이크아웃커피,서울특별시 강동구 길동,맛있어요!,,'커피가 맛있어요',2024-02-10,토요일,11
9,포도267,665,13,쉬즈네일,"네일아트,네일샵",서울특별시 강동구 길동,"몰랐는데 n년째다니는 쉬즈네일, 동네에 있어줘서 감사한 네일맛집 입니다. 재방문은 ...",None,"'시술이 꼼꼼해요', '원하는 디자인을 잘해줘요', '디자인 추천을 잘해줘요', '...",2024-02-02,금요일,3


In [185]:
# 리셋
data_list = []
del user_df

# 위 내용을 반복문으로 한번에 하기 

# 여기서부터 보면 됨

In [84]:
# 정보를 저장할 데이터프레임을 만든다. 
user_df = pd.DataFrame()

# 데이터프레임에 컬럼을 넣는다. 
new_columns = ['아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수']
for column in new_columns:
    user_df[column] = None 

user_df

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,재방문횟수


In [69]:
# 네이버에서 찾아온 리뷰어들의 주소 
# myplace url
user_list = ['https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2',
'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2',
# 'https://m.place.naver.com/my/5bd2f3682db9d1bdd38eedca/review?v=2', # 얘는 에러나서 주석처리 (맨 처음 테마가 있으니까 에러남)
'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2',
'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2',
'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2',
'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2']

In [92]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 만약 저게 에러나면 driver = webdriver.Chrome()


# url을 하나씩 가져와서 리뷰를 긁어온다. 
for url in user_list : 

    try : # 에러날까봐 try로 감쌈 


        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수 (위에 있는데 안봐도 괜찮음)
        user_data = user_profile(url)
        user_data_df = pd.DataFrame([user_data], index=[0])

        # User의 아이디, 리뷰수, 팔로워를 가져와서 user_profile_df에 concat으로 합쳐준다.
        if user_profile_df.index[-1] > 0: # 만약 User의 DataFrame이 이미 존재한다면, 기존 DataFrame에 추가
            user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
            user_profile_df = user_profile_df.drop_duplicates() # 중복제거
        else:
            user_profile_df = user_data_df 


        # 본격적으로 리뷰 긁어오기 

        # 맨 처음 게시물 클릭 
        button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
        button.click()
        time.sleep(5) # 기다려줌 

        # data를 담을 빈 리스트 선언
        data_list = [] 



        # # 반복해서 스크롤하기 - 에러남 
        # 아래 반복문의 구조는 현재 위치와 스크롤을 한 위치가 다르면 스크롤을 최대 5번까지 해라. 만약 같아지면 (더 리뷰가 없으면) break 
        # n = 0 
        # while n < 5 :
        #     last_scroll_position = driver.execute_script("return window.scrollY")
        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #     time.sleep(2)
        #     current_scroll_position = driver.execute_script("return window.scrollY")
            
        #     # 스크롤 위치에 변화가 없으면 종료
        #     if current_scroll_position == last_scroll_position:
        #         break
            
        #     # 현재 스크롤 위치를 다음 비교를 위해 저장
        #     last_scroll_position = current_scroll_position

        #     n += 1



        # 얘는 에러 안나서 일단 이렇게 스크롤하고 있음 
        # 10번 스크롤을 반복하고 요소를 찾을 때까지 기다렸다가 찾으면 user_review_elements 에 담아라
        # 페이지 아래로 스크롤 몇번
        page_down = 10

        # 페이지 스크롤 횟수만큼 반복
        for _ in range(page_down):

            # 대기시간 5초 
            wait = WebDriverWait(driver, 5)

            # 요소를 찾을 때 대기 조건 추가
            user_review_elements = wait.until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
            )

            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

            html = driver.page_source
            soup = BeautifulSoup(driver.page_source, 'html.parser')




            # 본격적으로 리뷰에서 정보 가져오기

            for reviews_elements in user_review_elements:

                i = 0 # 저 밑에서 카테고리와 주소 구분할 때 사용(안중요)

                # class _1QGRWW 를 가져온다. 
                restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
                
                if bool(restaurant_elements): # 정보가 없는 경우가 있음

                    # 매장명 찾기
                    restaurant_name = restaurant_elements[0].text  

            #<--------------------------------------------------------------------------------------------------->
                    # 카테고리와 주소 찾기 (카테고리와 주소가 붙어서 써있음 짜증)
                    category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                    category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
                    
                    # span 태그 안에 있는 텍스트 가져와서 카테고리와 주소로 나눔 
                    span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                    category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                    location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                    i += 1 # 근데 다시보니까 이거 왜 했더라 싶고? 

            #<--------------------------------------------------------------------------------------------------->
                    # 리뷰 찾기 
                    review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문
                    sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') # 세부정보 <- 있을때도 있고 없을때도 있음 
                    
                    if bool(sub_info):
                        sub_info=sub_info[0].text
                    else:
                        sub_info=None

            #<--------------------------------------------------------------------------------------------------->
                    # 태그 찾기 
                    # 일정 개수가 넘어가면 리뷰가 숨겨져 더보기를 눌러야함  
                    reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트

                    # 여긴 이정훈이 짜줌 
                    command=False
                    for x in reactions_elements:
                        if x.get_attribute("role"): # 리액션 숨김 없애기
                            x.click() # 더보기 클릭하기 
                            command=True
                        else:
                            continue
                    if command:
                        reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
                    
                    # reactions가 1~5개이므로 리스트로 묶어서 반환함
                    reactions=list(map(lambda x:x.text,reactions_elements))
                    # 리스트를 벗김 
                    reactions = str(reactions).replace('[', '').replace(']', '')
                    


            #<--------------------------------------------------------------------------------------------------->
                    # 방문 일자, 재방문 이력 찾기
                    date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                    date=[x.text for x in date_info]

                    # 한번에 묶여있어서 요일/날짜/방문횟수로 나눔 
                    if date:
                        day = date[0].split('\n')[0]
                        if day[-1:].isdigit(): # 요일이 없으면 
                            weekday = day[-1:]
                        else:
                            weekday = ""
                        day = day[:-2]
                        revisit = int(date[1].replace('번째 방문', ''))

                
                    # 정보 추가
                    data_dict = {
                            '아이디' : user_data['아이디'],
                            '리뷰' : user_data['리뷰'],
                            '팔로워' : user_data['팔로워'],
                            '매장명': restaurant_name,
                            '카테고리': category,
                            '주소' : location,
                            '리뷰 내용': review_text,
                            "세부정보" : sub_info,
                            '태그': reactions,
                            '방문일자': day,
                            '방문요일' : weekday,
                            '재방문횟수': revisit}
                    data_list.append(data_dict)

            # 여기가 에러 개많이 나는 부분인데 챗지피티놈 가만안둬 
            # user_df에 정보가 있으면 concat을 하고 없으면 (처음 시작하는거라면) 데이터프레임에 집어넣어라 라는 명령
            if 'user_df' in locals():
                user_df = pd.concat([user_df, pd.DataFrame(data_list)], ignore_index=True)
            else:
                user_df = pd.DataFrame(data_list)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue
                
user_df


{'아이디': 'xll****', '리뷰': 983, '팔로잉': 0, '팔로워': 75, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}
에러 메시지: list index out of range
{'아이디': '맛있는거 먹으려고 운동함', '리뷰': 534, '팔로잉': 2, '팔로워': 41, '주소': 'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2'}
{'아이디': '참참s', '리뷰': 495, '팔로잉': 0, '팔로워': 74, '주소': 'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2'}
에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3P-5HQ"}
  (Session info: chrome=121.0.6167.161)

{'아이디': 'JUDY0725', '리뷰': 1927, '팔로잉': 0, '팔로워': 95, '주소': 'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2'}
{'아이디': '초코피스타치오', '리뷰': 159, '팔로잉': 6, '팔로워': 24, '주소': 'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2'}
{'아이디': 'sodadada7', '리뷰': 179, '팔로잉': 1, '팔로워': 0, '주소': 'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2'}


,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,재방문횟수,방문요일
0,xll****,983,75,매우매오 강남역별관,"육류,고기요리",서울특별시 강남구 역삼동,구성이 좋아요! 끓이면서 따끈하게 먹을 수 있어 좋아요~,이용 방법예약 없이대기 시간바로 입장동행친구,'메뉴 구성이 알차요',2.13,1,
1,맛있는거 먹으려고 운동함,534,41,메디큐브의원 강남신논현역,피부과,서울특별시 서초구 서초동,"강남역 피부과입니다! 신논현역, 강남역에서 도보로 10분 내로 방문할 수 있어서 접...",이용 방법예약 후대기 시간바로 입장,,2.5,3,
2,참참s,495,74,우몽블랙 판교점,"육류,고기요리",경기도 성남시 분당구 백현동,고급스러운 프라이빗한 룸에서 한우를 직접 맛있게 구워주셔서 너무 특별한 식사였어요....,이용 방법예약 후대기 시간바로 입장목적친목동행친척·형제,"'직접 잘 구워줘요', '음식이 맛있어요', '고기 질이 좋아요', '양이 많아요'...",2.11,1,
3,JUDY0725,1927,95,옵스 마린시티점,베이커리,부산광역시 해운대구 우동,붐비지 않아서 쾌적하게 빵을 고를 수 있었어요! \n친절하게 설명해주시고 선물세트가...,이용 방법예약 없이대기 시간10분 이내목적나들이동행친구,"'빵이 맛있어요', '가성비가 좋아요', '디저트가 맛있어요', '특별한 메뉴가 있...",2.12,1,
4,초코피스타치오,159,24,메종 아카이,돈가스,서울특별시 종로구 명륜2가,혜화 샤브샤브 맛집 메종아카이 일본풍 아기자기한 분위기와 깔끔하고 정갈한 플레이팅이...,None,"'음식이 맛있어요', '특별한 메뉴가 있어요', '인테리어가 멋져요', '단체모임 ...",2.10,1,
...,...,...,...,...,...,...,...,...,...,...,...,...
4791,sodadada7,179,0,오생,"오뎅,꼬치",대구광역시 달서구 성당동,살면서 처음본 하이볼 타워가 있었던 대구 서부 정류장 맛집이에요😉ㅋㅋㅋㅋ 여섯잔 분...,None,"'술이 다양해요', '특별한 메뉴가 있어요', '대화하기 좋아요', '친절해요'",23.7.7,1,
4792,sodadada7,179,0,복골상회,없음,없음,고기 다 구워 주시니까 너무 편리하고 맛있어요~~~~\n채끝살 너무 부드러워요 ! ...,None,"'음식이 맛있어요', '고기 질이 좋아요', '매장이 청결해요', '직접 잘 구워줘요'",23.7.1,1,
4793,sodadada7,179,0,탑골프아카데미,골프연습장,서울특별시 송파구 잠실동,"3개월 등록해서 레슨 30분으로 10번 들었어요, 일주일만에 초고속으로 배워서 라운...",None,"'시설이 깔끔해요', '타석이 넓어요', '최신 장비를 사용해요', '친절해요', ...",23.6.17,1,
4794,sodadada7,179,0,제주노을 참숯불집,한식,서울특별시 송파구 방이동,다른 거 다 필요없고 일단 고기가 진짜 맛있습니다,None,"'친절해요', '음식이 맛있어요'",23.6.5,1,


In [93]:
# 얜 또 왜 저장이 안돼... 
# user의 프로필 정보와 리뷰 정보를 각각 저장하고 싶었음 
user_profile_df

,아이디,리뷰,팔로잉,팔로워,주소
0,sodadada7,179,1,0,https://m.place.naver.com/my/5e27e4ec8f87a842b...


저장하기

In [16]:
# 유저의 리뷰
user_df.to_excel('user_df.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('user_profile_df.xlsx', index=False)